In [ ]:
# This file is for creating dictionary of mapping between fine labels ranging from 0 to 99 to labels from 0 to 4
# first I divide by parent category, then in each of the 20 parents I map fine labels from 0 to 4

In [4]:
from __future__ import print_function
from PIL import Image
import os
import os.path
import errno
import numpy as np
import sys
import torch
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity
import torchvision.transforms as tt
from torchvision import datasets, transforms
import torchvision

class CIFAR10(data.Dataset):
    """`CIFAR10 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.
    Args:
        root (string): Root directory of dataset where directory
            ``cifar-10-batches-py`` exists.
        train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
    """
    base_folder = 'cifar-10-batches-py'
    url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    filename = "cifar-10-python.tar.gz"
    tgz_md5 = 'c58f30108f718f92721af3b95e74349a'
    train_list = [
        ['data_batch_1', 'c99cafc152244af753f735de768cd75f'],
        ['data_batch_2', 'd4bba439e000b95fd0a9bffe97cbabec'],
        ['data_batch_3', '54ebc095f3ab1f0389bbae665268c751'],
        ['data_batch_4', '634d18415352ddfa80567beed471001a'],
        ['data_batch_5', '482c414d41f54cd18b22e5b47cb7c3cb'],
    ]

    test_list = [
        ['test_batch', '40351d587109b95175f43aff81a1287e'],
    ]

    meta = {
        "filename": "batches.meta",
        "key": "label_names",
        "md5": "5ff9c542aee3614f3951f8cda6e48888",
    }


    def __init__(self, root, train=True,
                 transform=None, target_transform=None,
                 download=False, coarse=False):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set
        self.coarse = coarse

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

        # now load the picked numpy arrays
        if self.train:
            self.train_data = []
            self.train_labels = []
            self.train_coarse_labels = []
            for fentry in self.train_list:
                f = fentry[0]
                file = os.path.join(self.root, self.base_folder, f)
                fo = open(file, 'rb')
                if sys.version_info[0] == 2:
                    entry = pickle.load(fo)
                else:
                    entry = pickle.load(fo, encoding='latin1')
                self.train_data.append(entry['data'])
                if 'labels' in entry:
                    self.train_labels += entry['labels']
                else:
                    self.train_labels += entry['fine_labels']
                    if self.coarse:
                        self.train_coarse_labels += entry['coarse_labels']
                fo.close()

            self.train_data = np.concatenate(self.train_data)
            self.train_data = self.train_data.reshape((50000, 3, 32, 32))
            self.train_data = self.train_data.transpose((0, 2, 3, 1))  # convert to HWC
        else:
            f = self.test_list[0][0]
            file = os.path.join(self.root, self.base_folder, f)
            fo = open(file, 'rb')
            if sys.version_info[0] == 2:
                entry = pickle.load(fo)
            else:
                entry = pickle.load(fo, encoding='latin1')
            self.test_data = entry['data']

            if 'labels' in entry:
                self.test_labels = entry['labels']
            else:
                self.test_labels = entry['fine_labels']
                if self.coarse:
                    self.test_coarse_labels = entry['coarse_labels']
            fo.close()
            self.test_data = self.test_data.reshape((10000, 3, 32, 32))
            self.test_data = self.test_data.transpose((0, 2, 3, 1))  # convert to HWC

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
            if self.coarse:
                coarse_target = self.train_coarse_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]
            if self.coarse:
                coarse_target = self.test_coarse_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)
        if not self.coarse:
            return img, target
        else:
            return img, target, coarse_target

    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

    def _check_integrity(self):
        root = self.root
        for fentry in (self.train_list + self.test_list):
            filename, md5 = fentry[0], fentry[1]
            fpath = os.path.join(root, self.base_folder, filename)
            if not check_integrity(fpath, md5):
                return False
        return True

    def download(self):
        import tarfile

        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        root = self.root
        download_url(self.url, root, self.filename, self.tgz_md5)

        # extract file
        cwd = os.getcwd()
        tar = tarfile.open(os.path.join(root, self.filename), "r:gz")
        os.chdir(root)
        tar.extractall()
        tar.close()
        os.chdir(cwd)


class CIFAR100(CIFAR10):
    """`CIFAR100 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.
    This is a subclass of the `CIFAR10` Dataset.
    """
    base_folder = 'cifar-100-python'
    url = "https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz"
    filename = "cifar-100-python.tar.gz"
    tgz_md5 = 'eb9058c3a382ffc7106e4002c42a8d85'
    train_list = [
        ['train', '16019d7e3df5f24257cddd939b257f8d'],
    ]

    test_list = [
        ['test', 'f0ef6b0ae62326f3e7ffdfab6717acfc'],
    ]


In [5]:
transform_train = tt.Compose([tt.RandomCrop(32, padding=4,padding_mode='reflect'), 
                         tt.RandomHorizontalFlip(), 
                         tt.ToTensor(), 
                         #tt.Normalize(mean,std,inplace=True) 
                         ]
                         )

In [6]:
train_data = CIFAR100('./data', train=True,
                 transform=transform_train,
                 download=True, coarse=True)

Files already downloaded and verified


In [7]:
demo_loader = torch.utils.data.DataLoader(train_data, batch_size=1200)

demo_batch = next(iter(demo_loader))
img, fine_labels, course_labels = demo_batch
fine_labels=fine_labels.numpy()
course_labels=course_labels.numpy()

parent_to_child_class={}

for fine, coarse in zip(fine_labels, course_labels):
    if coarse not in parent_to_child_class:
        parent_to_child_class[coarse]=[fine]
    elif fine not in parent_to_child_class[coarse]:
        parent_to_child_class[coarse].append(fine)
        parent_to_child_class[coarse].sort()

print(len(parent_to_child_class.keys()))

count=0
for x in parent_to_child_class.values():
    count+=len(x)
print(count)

print(parent_to_child_class.keys())

20
100
dict_keys([11, 15, 4, 14, 1, 5, 18, 3, 10, 17, 2, 9, 8, 16, 6, 12, 19, 7, 13, 0])


In [8]:
parent_to_child_class.items()

dict_items([(11, [15, 19, 21, 31, 38]), (15, [27, 29, 44, 78, 93]), (4, [0, 51, 53, 57, 83]), (14, [2, 11, 35, 46, 98]), (1, [1, 32, 67, 73, 91]), (5, [22, 39, 40, 86, 87]), (18, [8, 13, 48, 58, 90]), (3, [9, 10, 16, 28, 61]), (10, [23, 33, 49, 60, 71]), (17, [47, 52, 56, 59, 96]), (2, [54, 62, 70, 82, 92]), (9, [12, 17, 37, 68, 76]), (8, [3, 42, 43, 88, 97]), (16, [36, 50, 65, 74, 80]), (6, [5, 20, 25, 84, 94]), (12, [34, 63, 64, 66, 75]), (19, [41, 69, 81, 85, 89]), (7, [6, 7, 14, 18, 24]), (13, [26, 45, 77, 79, 99]), (0, [4, 30, 55, 72, 95])])

In [9]:
parent_to_child_class[0]

[4, 30, 55, 72, 95]

In [10]:
parent_to_child_class_from_0_to_5={}

for parent, children in parent_to_child_class.items():
    print(parent)
    print(children)
    test_keys = children.copy()
    test_values = [0, 1, 2, 3 ,4]

    # Printing original keys-value lists
    print("Original values list is : " + str(test_keys))
    print("New value list is : " + str(test_values))

    # using naive method
    # to convert lists to dictionary
    res = {}
    for key in test_keys:
        for value in test_values:
            res[int(key)] = value
            test_values.remove(value)
            break
    parent_to_child_class_from_0_to_5[int(parent)]=res

print(str(parent_to_child_class_from_0_to_5))

11
[15, 19, 21, 31, 38]
Original values list is : [15, 19, 21, 31, 38]
New value list is : [0, 1, 2, 3, 4]
15
[27, 29, 44, 78, 93]
Original values list is : [27, 29, 44, 78, 93]
New value list is : [0, 1, 2, 3, 4]
4
[0, 51, 53, 57, 83]
Original values list is : [0, 51, 53, 57, 83]
New value list is : [0, 1, 2, 3, 4]
14
[2, 11, 35, 46, 98]
Original values list is : [2, 11, 35, 46, 98]
New value list is : [0, 1, 2, 3, 4]
1
[1, 32, 67, 73, 91]
Original values list is : [1, 32, 67, 73, 91]
New value list is : [0, 1, 2, 3, 4]
5
[22, 39, 40, 86, 87]
Original values list is : [22, 39, 40, 86, 87]
New value list is : [0, 1, 2, 3, 4]
18
[8, 13, 48, 58, 90]
Original values list is : [8, 13, 48, 58, 90]
New value list is : [0, 1, 2, 3, 4]
3
[9, 10, 16, 28, 61]
Original values list is : [9, 10, 16, 28, 61]
New value list is : [0, 1, 2, 3, 4]
10
[23, 33, 49, 60, 71]
Original values list is : [23, 33, 49, 60, 71]
New value list is : [0, 1, 2, 3, 4]
17
[47, 52, 56, 59, 96]
Original values list is : [

In [11]:
parent_to_child_class_from_0_to_5[3]

{9: 0, 10: 1, 16: 2, 28: 3, 61: 4}

In [12]:
import json

with open("parent_to_child_class_from_0_to_5.json", "w") as outfile: 
    json.dump(parent_to_child_class_from_0_to_5, outfile)

In [13]:
f = open('parent_to_child_class_from_0_to_5.json')
data = json.load(f)
print(data)

{'11': {'15': 0, '19': 1, '21': 2, '31': 3, '38': 4}, '15': {'27': 0, '29': 1, '44': 2, '78': 3, '93': 4}, '4': {'0': 0, '51': 1, '53': 2, '57': 3, '83': 4}, '14': {'2': 0, '11': 1, '35': 2, '46': 3, '98': 4}, '1': {'1': 0, '32': 1, '67': 2, '73': 3, '91': 4}, '5': {'22': 0, '39': 1, '40': 2, '86': 3, '87': 4}, '18': {'8': 0, '13': 1, '48': 2, '58': 3, '90': 4}, '3': {'9': 0, '10': 1, '16': 2, '28': 3, '61': 4}, '10': {'23': 0, '33': 1, '49': 2, '60': 3, '71': 4}, '17': {'47': 0, '52': 1, '56': 2, '59': 3, '96': 4}, '2': {'54': 0, '62': 1, '70': 2, '82': 3, '92': 4}, '9': {'12': 0, '17': 1, '37': 2, '68': 3, '76': 4}, '8': {'3': 0, '42': 1, '43': 2, '88': 3, '97': 4}, '16': {'36': 0, '50': 1, '65': 2, '74': 3, '80': 4}, '6': {'5': 0, '20': 1, '25': 2, '84': 3, '94': 4}, '12': {'34': 0, '63': 1, '64': 2, '66': 3, '75': 4}, '19': {'41': 0, '69': 1, '81': 2, '85': 3, '89': 4}, '7': {'6': 0, '7': 1, '14': 2, '18': 3, '24': 4}, '13': {'26': 0, '45': 1, '77': 2, '79': 3, '99': 4}, '0': {'4':